# D-REX

Minimal procedure:
1. Set input sequence
2. Calculate prior, given
    1. an input sequence to use
    2. parameters, including
        1. distribution (default: gaussian, options: gaussian, lognormal, gmm, poisson)
        2. D (default: 1, 50 for poisson)
3. Add prior to parameters, and optionally set further parameters
    1. hazard (default: 0.01)
    2. obsnz (default: 0.0)
    3. memory (default: inf)
    4. maxhyp (default: inf)
    5. Undocumented: predscale (default: 0.001)
4. Run model, optionally:
    1. Calculate belief dynamics (post_DREX_beliefdynamics.m)
    2. Calculate change decisions (post_DREX_changedecision.m)
    3. Calculate predictive distribution (post_DREX_prediction.m)
6. Interpret results